<h1> IPython Parallel Push/Execute/Pull Demo</h1>
<p> This notebook performs a very simple dot product of the same two random (square) matrices on all ipengines. It then gathers the results from all ipengines and confirms that all results are the same. This mostly demos how the push, execute, and pull pipeline works in the IPython Parallel client.</p>

<h1> 1. Allocate random matrices locally </h1>

In [16]:
import numpy as np

In [17]:
N = 2000

In [18]:
a = np.random.randn(N,N)

In [19]:
b = np.random.randn(N,N)

<h1> 2. Compute their dot product locally </h1>

In [20]:
c = np.dot(a,b)

<h1> 3. Initialize the IPython Parallel Client</h1>

In [21]:
from ipyparallel import Client

In [22]:
cl = Client()

In [23]:
cl.ids

[0, 1, 2, 3, 4, 5, 6, 7]

<h1> 4. Fetch a 'view' from the IPython parallel client</h1>

In [24]:
v = cl[:]

<h1> 5. Import NumPy on all IPEngines </h1>

In [25]:
v.execute('import numpy as np', block =True)

<AsyncResult: execute:finished>

<h1> 6. Push both random matrices to all IPEngines</h1>

In [26]:
v.push(dict(a=a, b=b), block=True)

[None, None, None, None, None, None, None, None]

<h1> 7. Compute dot product on all IPEngines</h1>

In [34]:
r = v.execute('c = np.dot(a,b); print (c)', block=True)

In [35]:
r.display_outputs()

[stdout:0] 
[[ -12.5152144    -1.69567106  -18.59301171 ...,   12.68383101
    -7.69762498   53.85882056]
 [  19.84973481   18.9781437   -46.46425558 ...,   66.53682964
   -36.87390277  -46.86621431]
 [ -19.82512791    3.98374177  -35.80323397 ...,    2.74710432
    35.08259466    7.05908044]
 ..., 
 [ -46.64011791   22.86409286    7.15741079 ...,   64.14027678
   -29.03762211  -24.98852555]
 [  48.34118734   23.08994966  -56.14982631 ...,    4.25435926
    50.86567914   -1.5525937 ]
 [   9.26765535 -101.54860375   23.91174727 ...,  -74.39503164
    49.21603507   -4.88055697]]
[stdout:1] 
[[ -12.5152144    -1.69567106  -18.59301171 ...,   12.68383101
    -7.69762498   53.85882056]
 [  19.84973481   18.9781437   -46.46425558 ...,   66.53682964
   -36.87390277  -46.86621431]
 [ -19.82512791    3.98374177  -35.80323397 ...,    2.74710432
    35.08259466    7.05908044]
 ..., 
 [ -46.64011791   22.86409286    7.15741079 ...,   64.14027678
   -29.03762211  -24.98852555]
 [  48.34118734   23.

<h1> 8. Pull the results from all IPEngines</h1>

In [31]:
dot = v.pull('c', block=True)

In [32]:
print (dot)

[array([[ -12.5152144 ,   -1.69567106,  -18.59301171, ...,   12.68383101,
          -7.69762498,   53.85882056],
       [  19.84973481,   18.9781437 ,  -46.46425558, ...,   66.53682964,
         -36.87390277,  -46.86621431],
       [ -19.82512791,    3.98374177,  -35.80323397, ...,    2.74710432,
          35.08259466,    7.05908044],
       ..., 
       [ -46.64011791,   22.86409286,    7.15741079, ...,   64.14027678,
         -29.03762211,  -24.98852555],
       [  48.34118734,   23.08994966,  -56.14982631, ...,    4.25435926,
          50.86567914,   -1.5525937 ],
       [   9.26765535, -101.54860375,   23.91174727, ...,  -74.39503164,
          49.21603507,   -4.88055697]]), array([[ -12.5152144 ,   -1.69567106,  -18.59301171, ...,   12.68383101,
          -7.69762498,   53.85882056],
       [  19.84973481,   18.9781437 ,  -46.46425558, ...,   66.53682964,
         -36.87390277,  -46.86621431],
       [ -19.82512791,    3.98374177,  -35.80323397, ...,    2.74710432,
          35.08

<h1> 9. Check remote results against local results </h1>

In [36]:
assert np.allclose(c, dot[0])

In [41]:
for i in range(7):
    assert np.allclose(dot[i], dot[i+1])